In [1]:
from gas import GASModel
from utils import generate_timeseries, generate_dataset

import torch
import torch.nn as nn
import torch.optim as optim

import plotly.graph_objects as go
import numpy as np

# Generate time series and dataset

In [2]:
GROWING = True
t, y = generate_timeseries(GROWING)

In [3]:
# split the timeseries
MAX_INDEX_TRAIN = 1500
MAX_INDEX_TEST = 2500
y_train = y[:MAX_INDEX_TRAIN]
y_test = y[MAX_INDEX_TRAIN: MAX_INDEX_TEST]

In [4]:
# generate the dataset as tensor of shape 
# (n_timeseries, ts_length, n_features) for inputs
# (n_timeseries, el_to_predict) for labels
TS_LENGTH = 200
EL_TO_PREDICT = 50
y_train, lab_train = generate_dataset(y_train, TS_LENGTH, EL_TO_PREDICT)
y_test, lab_test = generate_dataset(y_test, TS_LENGTH, EL_TO_PREDICT)

y_train.shape, lab_train.shape, y_test.shape, lab_test.shape

(torch.Size([26, 200, 1]),
 torch.Size([26, 50]),
 torch.Size([16, 200, 1]),
 torch.Size([16, 50]))

Duplicate the time series to simulate more than one feature

In [5]:
y_train = torch.cat((y_train, y_train), dim=2)
y_test = torch.cat((y_test, y_test), dim=2)

y_train.shape, y_test.shape

(torch.Size([26, 200, 2]), torch.Size([16, 200, 2]))

# Test GAS

In [6]:
# initialize gas params
eta_mu = 0.999
eta_sigma2 = 0.999

# the encoder of the time series is just a flattener of the time dimension
ts_encoder = nn.Flatten()
# the output model is a feedforward network
# the output of ts_encoder is (batch, ts_length * n_features)
# the additional info is (batch, ts_length * n_features * 2)
input_dim = TS_LENGTH*y_train.shape[2] + 2*TS_LENGTH*y_train.shape[2]  

HID_SIZE_1 = 100
HID_SIZE_2 = 100
output_model = nn.Sequential(nn.Linear(input_dim, HID_SIZE_1),
                                nn.ReLU(),
                                nn.Linear(HID_SIZE_1, HID_SIZE_2),
                                nn.ReLU(),
                                nn.Linear(HID_SIZE_2, EL_TO_PREDICT)
                                )

model = GASModel(ts_encoder, eta_mu, eta_sigma2, output_model)

# Define train, evaluate and plot result functions

In [7]:
def train_model(model, criterion, optimizer, epochs, y_train, lab_train):

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0

        for inputs, labels in zip(y_train, lab_train):
            # the first dimension must be batch_size (i.e. 1)
            inputs = inputs.unsqueeze(0)
            labels = labels.unsqueeze(0)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('[%d] loss: %.10f' %
                (epoch + 1, running_loss / y_train.shape[0]))

    print('Finished Training')

    return model


def evaluate_model(model, y_all):

    model.eval()
    y_pred = []
    for inputs in y_all:
        inputs = inputs.unsqueeze(0)    # first dimension must be batch_size
        outputs = model(inputs.float())
        outputs = outputs.squeeze()   # no need for batch_size here
        y_pred.append(outputs.detach().numpy())

    y_pred = np.array(y_pred)
    #y_pred = y_pred.reshape(-1)

    return y_pred


def plot_results(t, y, y_pred, max_index_test, max_index_train, ts_length):
    #plot with plotly with a line where the training set ends
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t[0:max_index_test], y=y, mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=t[ts_length:max_index_test], y=y_pred, mode='lines', name='Predicted'))
    fig.add_trace(go.Scatter(x=[t[max_index_train], t[max_index_train]], y=[-20, 150], mode='lines', name='Training Set End'))
    fig.update_layout(title='Dampened Sinusoid', xaxis_title='Time (s)', yaxis_title='Amplitude')
    fig.show()

# Run the experiment

In [8]:
# train the model
criterion = nn.MSELoss()
LR = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LR)
epochs = 400

model = train_model(model, criterion, optimizer, epochs, y_train, lab_train)

[1] loss: 2297.2544790415
[2] loss: 1563.6168072040
[3] loss: 676.5683758075
[4] loss: 225.9161547147
[5] loss: 74.0843940148
[6] loss: 45.3502423947
[7] loss: 41.0835571289
[8] loss: 39.5205024939
[9] loss: 38.2316496739
[10] loss: 36.9689821463
[11] loss: 35.7126024503
[12] loss: 34.4650472494
[13] loss: 33.2303079275
[14] loss: 32.0040689432
[15] loss: 30.7574976408
[16] loss: 29.5505145880
[17] loss: 28.3560059437
[18] loss: 27.1812880773
[19] loss: 26.0101078107
[20] loss: 24.8673463785
[21] loss: 23.8150001306
[22] loss: 22.6675180288
[23] loss: 21.5969421864
[24] loss: 20.5593825670
[25] loss: 19.5579201992
[26] loss: 18.5751224206
[27] loss: 17.6343874656
[28] loss: 16.7269352766
[29] loss: 15.8567844996
[30] loss: 15.0275296431
[31] loss: 14.2424933544
[32] loss: 13.4936678868
[33] loss: 12.7812279371
[34] loss: 12.1162852966
[35] loss: 11.5109310242
[36] loss: 10.9254116187
[37] loss: 10.3854576533
[38] loss: 9.8921684394
[39] loss: 9.4294055975
[40] loss: 9.0082553350
[41] l

In [9]:
# evaluate the model on the entire dataset
y_all, lab_all = generate_dataset(y, TS_LENGTH, EL_TO_PREDICT)

# memento: we must have two features
y_all = torch.cat((y_all, y_all), dim=2)

y_pred = evaluate_model(model, y_all)
y_all.shape, lab_all.shape, y_pred.shape

(torch.Size([46, 200, 2]), torch.Size([46, 50]), (46, 50))

In [10]:
y_pred = y_pred.reshape(-1)
plot_results(t, y, y_pred, MAX_INDEX_TEST, MAX_INDEX_TRAIN, TS_LENGTH)

In [11]:
#Compute the total error on the test set.
error = 0
for i in range(MAX_INDEX_TRAIN, MAX_INDEX_TEST):
    error = error + (y[i] - y_pred[i-TS_LENGTH])**2
error = error/(MAX_INDEX_TEST - MAX_INDEX_TRAIN)
print(error)

430.349877370244


# Test DAIN

In [12]:
MEAN_LR = 1e-06
STD_LR = 0.001
GATE_LR = 10

n_features = y_train.shape[-1]      # shape is (batch, ts_length, n_features)

# let's say we want two hidden layers
HIDDEN_DIM_1 = 100
HIDDEN_DIM_2 = 100     

output_dim = lab_train.shape[-1]    # shape is (batch, el_to_predict)

return_means = True

# we will simply flatten the time series and concatenate the mean vector
input_dim = y_train.shape[1] * n_features + n_features

In [13]:
from dain import DAINLayer

class DAINWrapper(nn.Module):

    def __init__(self, n_features, mode, MEAN_LR, STD_LR, GATE_LR, return_means):
        super(DAINWrapper, self).__init__()

        self.dain = DAINLayer(n_features, mode, MEAN_LR, STD_LR, GATE_LR, return_means)

    def forward(self, x):
        x, mu = self.dain(x)
        # flatten the output to get (batch, ts_length)
        x = x.reshape((x.shape[0], -1))
        # concatenate with mu, rememer mu's shape is (batch, n_features)
        x = torch.cat((x, mu), dim=1)
        return x

In [14]:
mode = 'adaptive_avg'

dain = DAINWrapper(n_features, mode, MEAN_LR, STD_LR, GATE_LR, return_means)

model = nn.Sequential(dain,
                      nn.Flatten(),
                      nn.Linear(input_dim, HIDDEN_DIM_1),
                      nn.ReLU(),
                      nn.Linear(HIDDEN_DIM_1, HIDDEN_DIM_2),
                      nn.ReLU(),
                      nn.Linear(HIDDEN_DIM_2, output_dim))
model

Sequential(
  (0): DAINWrapper(
    (dain): DAINLayer(
      (mean_layer): Linear(in_features=2, out_features=2, bias=False)
      (scaling_layer): Linear(in_features=2, out_features=2, bias=False)
      (gating_layer): Linear(in_features=2, out_features=2, bias=True)
    )
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=402, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=50, bias=True)
)

In [15]:
criterion = nn.MSELoss()
LR = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LR)
EPOCHS = 1000

train_model(model, criterion, optimizer, EPOCHS, y_train, lab_train)

[1] loss: 2494.1327796349
[2] loss: 2443.8587083083
[3] loss: 2355.6149432843
[4] loss: 2196.8469073956
[5] loss: 1935.5294165978
[6] loss: 1557.6757037823
[7] loss: 1107.2749487070
[8] loss: 709.0725238507
[9] loss: 426.5277566176
[10] loss: 238.5324324094
[11] loss: 140.5853583996
[12] loss: 90.3834204307
[13] loss: 61.7018638758
[14] loss: 44.1761629031
[15] loss: 32.7269075100
[16] loss: 25.1912847299
[17] loss: 20.1958266405
[18] loss: 16.6909011511
[19] loss: 14.1838790270
[20] loss: 12.2498731613
[21] loss: 10.7596983543
[22] loss: 9.6284380693
[23] loss: 8.7533700283
[24] loss: 8.0467251172
[25] loss: 7.4694206623
[26] loss: 6.9866347405
[27] loss: 6.5889149904
[28] loss: 6.2447680693
[29] loss: 5.9571071680
[30] loss: 5.7091372655
[31] loss: 5.4909922435
[32] loss: 5.3023293385
[33] loss: 5.1357270938
[34] loss: 4.9895364596
[35] loss: 4.8550653274
[36] loss: 4.7374572112
[37] loss: 4.6325625731
[38] loss: 4.5363557614
[39] loss: 4.4429519085
[40] loss: 4.3577446296
[41] loss:

Sequential(
  (0): DAINWrapper(
    (dain): DAINLayer(
      (mean_layer): Linear(in_features=2, out_features=2, bias=False)
      (scaling_layer): Linear(in_features=2, out_features=2, bias=False)
      (gating_layer): Linear(in_features=2, out_features=2, bias=True)
    )
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=402, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=50, bias=True)
)

In [16]:
# evaluate the model on the entire dataset
y_all, lab_all = generate_dataset(y, TS_LENGTH, EL_TO_PREDICT)

# memento: we must have two features
y_all = torch.cat((y_all, y_all), dim=2)

y_pred = evaluate_model(model, y_all)
y_pred.shape

(46, 50)

In [17]:
y_pred = y_pred.reshape(-1)
plot_results(t, y, y_pred, MAX_INDEX_TEST, MAX_INDEX_TRAIN, TS_LENGTH)

In [18]:
#Compute the total error on the test set
error = 0
for i in range(MAX_INDEX_TRAIN, MAX_INDEX_TEST):
    error = error + (y[i] - y_pred[i-TS_LENGTH])**2
error = error/(MAX_INDEX_TEST - MAX_INDEX_TRAIN)
print(error)

5028.705449277512
